In [2]:
def loadDataset():
        path = 'Task3_MNIST_Data'
        print('Loading Dataset...')
        train_x, train_y, test_x, test_y = [], [], [], []
        for i in range(10):
            for filename in glob.glob(path + '\\train\\' + str(i)+'\\*.png'):

                im=img.imread(filename)
                train_x.append(im)
                train_y.append(i)
        for i in range(10):
            for filename in glob.glob(path + '\\test\\' + str(i)+'\\*.png'):
                im=img.imread(filename)
                test_x.append(im)
                test_y.append(i)
        print('Dataset loaded...')
        return np.array(train_x), np.array(train_y), np.array(test_x),np.array(test_y)

In [3]:
x,y, X,Y = loadDataset()

Loading Dataset...
Dataset loaded...


In [4]:
n = x.T.reshape(60000,-1)   # short hand
n.shape

(60000, 784)

In [36]:
## Include libraries which may use in implementation
import numpy as np
import random
import sklearn.datasets as ds
import matplotlib.pyplot as plt
import glob
import pandas as pd
from matplotlib import image as img
# from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import seaborn as sns
import numpy as np


# Create a Neural_Network class
class Neural_Network(object):        
    def __init__(self,inputSize = 784 ,hiddenlayer = [128, 64], outputSize = 10 ):        
        # size of layers
        self.inputSize = inputSize
        self.outputSize = outputSize 
        self.hiddenLayer = hiddenlayer
        layers = [inputSize]+ hiddenlayer + [outputSize] 
#         print(layers)
        self.w =   np.array([np.random.rand(layers[i] , layers[i+1]) for i in  range(len(layers) -1)],dtype=object)
#         for i in self.w:
#             print(i.shape)
        self.der =  np.array([np.zeros((layers[i] , layers[i+1]), dtype='uint8') for i in  range(len(layers) -1)] , dtype=object)
#         print(type(self.der))
        self.act = np.array([np.zeros((layers[i])) for i in range(len(layers))] , dtype=object)
#         print(type(self.act))

    def mis(self, images):
        mi = images - images.mean()
        return mi
        
        
    def feedforward(self, X):
        self.act[0]= X
        inputs = X
        for i in range(len(self.w)):
#             print('here' , self.w[i].shape)
            if i < (len(self.w)-1):
                act = np.dot(inputs, self.w[i])
                outputs = self.sigmoid(act)
#                 print("sigmoid" , outputs)
                inputs = outputs
                self.act[i+1] = outputs
            else:
                act = np.dot(inputs, self.w[i])
                outputs = self.softmax(act)
#                 print(np.sum(outputs))
                inputs = outputs
                self.act[i+1] = outputs
#         print(self.act)
        return outputs
    
    def sigmoid(self, s):
        s = s.astype(float)
        return 1/(1+np.exp(-s))
    
    def sig_der(self,s):
        return (np.exp(-s))/((np.exp(-s)+1)**2)
    
    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0) # only difference
    
    def softmax_der(self,x):
        exps= np.exp(x - np.max(x))
        return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))

    
    def one_hot(self, Y):
        e = np.zeros((1 , 10) , np.float32)
        np.put(e,Y, 1.0)
        return e

    def backwardpropagate(self ,y, y_pred):
#         newY = np.apply_along_axis(self.one_hot, 1, y)
#         yy = (newY.reshape(len(newY), -1))
#         print(yy.shape , y_pred.shape)
        loss = (y - y_pred).mean()
#         print(loss)
        error = loss*self.softmax_der(self.act[3])
#         print(error.shape)
        self.der[2] = np.dot(error.T, self.act[2]) 
#         print("der2",self.der[2].shape)
        
        error = (np.dot(error, self.w[2].T)) * self.sig_der(self.act[2])
#         print(error.shape)
        self.der[1] = np.dot(error.T,self.act[1])
#         print(self.der[1].shape)
        
        error = (np.dot(error, self.w[1].T)) * self.sig_der(self.act[1])
        self.der[0] = np.dot(error.T,self.act[0])
#         print(self.der[0].shape)
        
        return self
    
    def weights_update(self, lr):

        #fixing the shape of self.der to align with self.w
        for i in range(len(self.der)):
            self.der[i] = self.der[i].T
            
        self.w = self.w + lr*self.der
        return self
    
    def decode(self, y , y_pred):
        y = np.argmax(y)
        y_pred = np.argmax(y_pred)
        return y,y_pred
    
    def accuracy(self, y, y_pred):
        ypred = np.argmax(y_pred, axis=1)[:,None]
        y = np.argmax(y, axis=1)[:,None]
#         print(y.shape , ypred.shape)
        return (y==ypred).mean()

    
    def loss(self, y, y_pred):
        loss = ((y - y_pred)**2).mean()
        return loss

    def train(self, trainX, trainY, epochs =5 , learningRate = 0.001, plot_err = True):
        epoch_loss = []
        epoch_acc = []
        for i in range(epochs):
            pred = self.feedforward(trainX)
            cost = (trainY-pred).mean()
            epoch_loss.append(cost)
            acc = self.accuracy(trainY,pred)
            epoch_acc.append(acc)
            print("epoch" ,i , "loss", cost , 'accuracy', acc)
            self.backwardpropagate(trainY,pred)
            self.weights_update(learningRate)
            
    def test(self,testX , testY,  learningRate = 0.001):
        pred = self.feedforward(testX)
        loss = (testY - pred).mean()
        acc = self.accuracy(testY,pred)
        self.backwardpropagate(testY,pred)
        self.weights_update(learningRate)
        print("Loss" , loss , "Accuracy " ,acc)
    

def main(x,y, X,Y):   
    data = x.T.reshape(60000,-1)  
    labels = y.reshape(-1,1)
    testX = X.T.reshape(10000,-1) 
    testY = Y.reshape(-1,1)
    #print(data.shape, labels.shape , testX.shape , testY.shape)
    
    model = Neural_Network()

    #mean image subtraction
    data = model.mis(data)
    
    #shuffle data for better results
    trainX, valX,  trainY , valY = train_test_split(data, labels, test_size=0.33, random_state=40, shuffle=True)
    print(trainX.shape, trainY.shape, valX.shape, valY.shape)


#     print((trainX[0]).reshape(-1,1).shape)
#     res = model.feedforward((trainX))
    
#     print("output res" , res.shape)
#     print(res)
    newY = np.apply_along_axis(model.one_hot, 1, trainY)
    yy = (newY.reshape(len(newY), -1))
    
#     der = model.backwardpropagate(yy ,res)
#     wts = model.weights_update(0.00001)
    
#     model.train(trainX , yy )
    model.test(testX , testY  , 0.0001)

#     print(model.accuracy(trainY, res))
if __name__ == '__main__':
    main(x,y,X,Y)

(40200, 784) (40200, 1) (19800, 784) (19800, 1)
Loss 4.443299999999998 Accuracy  0.0
